<a href="https://colab.research.google.com/github/PhuphaB/Data-Analytics/blob/main/Individual_Summative_Assessment_EFIMM0142_24_25(Draft).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1 (complete)


##Packages

In [2]:
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils
from pulp import GLPK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 57.8 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking 

##1a

In [31]:
import pulp
from pulp import LpVariable, LpProblem, LpStatus, lpSum, LpMinimize, value
import pandas as pd
# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Total_Cost"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nTotal Cost:", value(prob.objective))


Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       162.0       42.0
9    Feb       C       201.0      113.0
10   Mar       C       126.0       99.0
11   Apr       C        51.0        0.0

Total Cost: 26127.0


##1b

In [66]:
import pulp
from pulp import LpVariable, LpProblem, LpStatus, lpSum, LpMinimize, value
import pandas as pd

# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs with increased cost for Product B in March
production_cost = {
    'A': [20, 20, 20, 20],
    'B': [30, 30, 35, 30],  # Increased cost for Product B in March
    'C': [10, 10, 10, 10]
}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p][i] * x[f"{p}_{m}"] for p in products for i, m in enumerate(months)) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Total_Cost"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nTotal Cost:", value(prob.objective))


Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B       190.0      100.0
6    Mar       B         0.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       179.0       59.0
9    Feb       C        71.0        0.0
10   Mar       C       239.0       99.0
11   Apr       C        51.0        0.0

Total Cost: 26279.0


##1c

In [67]:
import pulp
from pulp import LpVariable, LpProblem, LpStatus, lpSum, LpMinimize, value
import pandas as pd

# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Total_Cost"

# Define demand constraints with increased demand for Product C in April
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 170]  # Increased demand for Product C in April by 20 units
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nTotal Cost:", value(prob.objective))



Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       182.0       62.0
9    Feb       C       201.0      133.0
10   Mar       C       126.0      119.0
11   Apr       C        51.0        0.0

Total Cost: 26357.0


##1d

In [69]:
import pulp
from pulp import LpVariable, LpProblem, LpStatus, lpSum, LpMinimize, value
import pandas as pd

# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Total_Cost"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Add constraint for total inventory holding cost not exceeding £500
prob += (
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months) <= 500,
    "Total_Holding_Cost"
)

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nTotal Cost:", value(prob.objective))


Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       162.0       42.0
9    Feb       C       201.0      113.0
10   Mar       C       126.0       99.0
11   Apr       C        51.0        0.0

Total Cost: 26127.0


#Question 2 (complete)

##2a

In [73]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, value

# Data from the problem
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City", "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"]
gallons_required = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]
cincinnati_cost = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_cost = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

# Production costs
cincinnati_production_cost = 1.20
oakland_production_cost = 1.65

# Total cost per gallon for each location from each facility
cincinnati_total_cost = [c + cincinnati_production_cost for c in cincinnati_cost]
oakland_total_cost = [o + oakland_production_cost for o in oakland_cost]

# Define the problem
prob = LpProblem("Minimise_Transportaion_Cost", LpMinimize)

# Define the decision variables
x = LpVariable.dicts("Route", [(i,j) for i in range(2) for j in range(11)], lowBound=0)

# Define the cost matrix (production + shipping costs)
cost = [cincinnati_total_cost, oakland_total_cost]

# Define the supply and demand constraints
supply = [500000, 500000]  # Assuming a capacity of 500,000 gallons for each plant

# Objective function: Minimize total cost
prob += lpSum([cost[i][j] * x[i,j] for i in range(2) for j in range(11)])

# Supply constraints
for i in range(2):
    prob += lpSum([x[i,j] for j in range(11)]) <= supply[i]

# Demand constraints
for j in range(11):
    prob += lpSum([x[i,j] for i in range(2)]) >= gallons_required[j]

# Solve the problem
prob.solve()

# Print the results with names and costs
results = []
total_gallons_cincinnati = total_gallons_oakland = total_cost = 0

for j in range(11):
    gallons_cincinnati = value(x[0,j])
    gallons_oakland = value(x[1,j])
    total_gallons = gallons_cincinnati + gallons_oakland
    cost_cincinnati = gallons_cincinnati * cincinnati_total_cost[j]
    cost_oakland = gallons_oakland * oakland_total_cost[j]
    total_location_cost = cost_cincinnati + cost_oakland

    results.append({
        "Location": locations[j],
        "From Cincinnati": gallons_cincinnati,
        "From Oakland": gallons_oakland,
        "Total Gallons": total_gallons,
        "Cost": total_location_cost
    })

    total_gallons_cincinnati += gallons_cincinnati
    total_gallons_oakland += gallons_oakland
    total_cost += total_location_cost

# Print the results in a formatted way
print("Results:")
print(f"{'Location':<15} {'From Cincinnati':<20} {'From Oakland':<15} {'Total Gallons':<15} {'Cost':<10}")
for result in results:
    print(f"{result['Location']:<15} {result['From Cincinnati']:<20} {result['From Oakland']:<15} {result['Total Gallons']:<15} {result['Cost']:<10.2f}")

print(f"\n{'Total':<15} {total_gallons_cincinnati:<20} {total_gallons_oakland:<15} {total_gallons_cincinnati + total_gallons_oakland:<15} {total_cost:<10.2f}")

Results:
Location        From Cincinnati      From Oakland    Total Gallons   Cost      
Santa Ana       0.0                  22418.0         22418.0         41921.66  
El Paso         6800.0               0.0             6800.0          13872.00  
Pendleton       39636.0              40654.0         80290.0         167460.64 
Houston         100447.0             0.0             100447.0        165737.55 
Kansas City     24570.0              0.0             24570.0         38329.20  
Los Angeles     0.0                  64761.0         64761.0         121103.07 
Glendale        0.0                  33689.0         33689.0         62998.43  
Jacksonville    68486.0              0.0             68486.0         105468.44 
Little Rock     148586.0             0.0             148586.0        228822.44 
Bridgeport      111475.0             0.0             111475.0        171671.50 
Sacramento      0.0                  112000.0        112000.0        201600.00 

Total           500000.0      

##2b

In [74]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, value

# Data from the problem
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City", "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"]
gallons_required = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]
cincinnati_cost = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_cost = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

# Production costs
cincinnati_production_cost = 1.20
oakland_production_cost = 1.65

# Total cost per gallon for each location from each facility
cincinnati_total_cost = [c + cincinnati_production_cost for c in cincinnati_cost]
oakland_total_cost = [o + oakland_production_cost for o in oakland_cost]

# Define the problem
prob = LpProblem("Minimise_Transportaion_Cost", LpMinimize)

# Define the decision variables
x = LpVariable.dicts("Route", [(i,j) for i in range(2) for j in range(11)], lowBound=0)

# Define the cost matrix (production + shipping costs)
cost = [cincinnati_total_cost, oakland_total_cost]

# Define the supply and demand constraints
supply = [500000, 500000]  # Assuming a capacity of 500,000 gallons for each plant

# Objective function: Minimize total cost
prob += lpSum([cost[i][j] * x[i,j] for i in range(2) for j in range(11)])

# Supply constraints
for i in range(2):
    prob += lpSum([x[i,j] for j in range(11)]) <= supply[i]

# Demand constraints
for j in range(11):
    prob += lpSum([x[i,j] for i in range(2)]) >= gallons_required[j]

# Solve the problem
prob.solve()

# Calculate the total cost for each location
total_costs = []
for j in range(11):
    gallons_cincinnati = value(x[0,j])
    gallons_oakland = value(x[1,j])
    cost_cincinnati = gallons_cincinnati * cincinnati_total_cost[j]
    cost_oakland = gallons_oakland * oakland_total_cost[j]
    total_location_cost = cost_cincinnati + cost_oakland
    total_costs.append(total_location_cost)

# Calculate the total cost for all locations
total_cost = sum(total_costs)

# Calculate the total gallons required
total_gallons_required = sum(gallons_required)

# Calculate the breakeven price per gallon
breakeven_price_per_gallon = total_cost / total_gallons_required

print(f"The breakeven price per gallon for Solutions Plus is {breakeven_price_per_gallon:.2f} USD.")

The breakeven price per gallon for Solutions Plus is 1.71 USD.


##2c

In [72]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, value

# Data from the problem
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City", "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"]
gallons_required = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]
cincinnati_cost = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_cost = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

# Production costs
cincinnati_production_cost = 1.20
oakland_production_cost = 1.65

# Total cost per gallon for each location from each facility
cincinnati_total_cost = [c + cincinnati_production_cost for c in cincinnati_cost]
oakland_total_cost = [o + oakland_production_cost for o in oakland_cost]

# Define the problem
prob = LpProblem("Minimise_Transportaion_Cost", LpMinimize)

# Define the decision variables
x = LpVariable.dicts("Route", [(i,j) for i in range(2) for j in range(11)], lowBound=0)

# Define the cost matrix (production + shipping costs)
cost = [cincinnati_total_cost, oakland_total_cost]

# Define the supply and demand constraints
supply = [500000, 500000]  # Assuming a capacity of 500,000 gallons for each plant

# Objective function: Minimize total cost
prob += lpSum([cost[i][j] * x[i,j] for i in range(2) for j in range(11)])

# Supply constraints
for i in range(2):
    prob += lpSum([x[i,j] for j in range(11)]) <= supply[i]

# Demand constraints
for j in range(11):
    prob += lpSum([x[i,j] for i in range(2)]) >= gallons_required[j]

# Solve the problem
prob.solve()

# Calculate the total cost for each location
total_costs = []
for j in range(11):
    gallons_cincinnati = value(x[0,j])
    gallons_oakland = value(x[1,j])
    cost_cincinnati = gallons_cincinnati * cincinnati_total_cost[j]
    cost_oakland = gallons_oakland * oakland_total_cost[j]
    total_location_cost = cost_cincinnati + cost_oakland
    total_costs.append(total_location_cost)

# Calculate the total cost for all locations
total_cost = sum(total_costs)

# Calculate the total gallons required
total_gallons_required = sum(gallons_required)

# Calculate the breakeven price per gallon
breakeven_price_per_gallon = total_cost / total_gallons_required

# Calculate the bid price with a 15% markup
markup_percentage = 0.15
bid_price_per_gallon = breakeven_price_per_gallon * (1 + markup_percentage)

print(f"The bid price per gallon for Solutions Plus with a 15% markup is {bid_price_per_gallon:.2f} USD.")

The bid price per gallon for Solutions Plus with a 15% markup is 1.96 USD.


##2d

In [75]:
import numpy as np
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, value

# Data from the problem
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City", "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"]
gallons_required = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]
cincinnati_cost = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_cost = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

# Production costs
cincinnati_production_cost = 1.20
oakland_production_cost = 1.65

# Define the problem
prob = LpProblem("Minimise_Transportaion_Cost", LpMinimize)

# Define the decision variables
x = LpVariable.dicts("Route", [(i,j) for i in range(2) for j in range(11)], lowBound=0)

# Define the cost matrix (production + shipping costs)
def calculate_total_costs(fuel_multiplier):
    cincinnati_total_cost = [(c + cincinnati_production_cost) * fuel_multiplier for c in cincinnati_cost]
    oakland_total_cost = [(o + oakland_production_cost) * fuel_multiplier for o in oakland_cost]
    return [cincinnati_total_cost, oakland_total_cost]

# Define the supply and demand constraints
supply = [500000, 500000]  # Assuming a capacity of 500,000 gallons for each plant

# Function to solve the problem and calculate costs
def solve_problem(fuel_multiplier):
    cost = calculate_total_costs(fuel_multiplier)
    prob = LpProblem("TransportationProblem", LpMinimize)
    prob += lpSum([cost[i][j] * x[i,j] for i in range(2) for j in range(11)])
    for i in range(2):
        prob += lpSum([x[i,j] for j in range(11)]) <= supply[i]
    for j in range(11):
        prob += lpSum([x[i,j] for i in range(2)]) >= gallons_required[j]
    prob.solve()
    total_costs = []
    for j in range(11):
        gallons_cincinnati = value(x[0,j])
        gallons_oakland = value(x[1,j])
        cost_cincinnati = gallons_cincinnati * cost[0][j]
        cost_oakland = gallons_oakland * cost[1][j]
        total_location_cost = cost_cincinnati + cost_oakland
        total_costs.append(total_location_cost)
    total_cost = sum(total_costs)
    return total_cost

# Simulate different fuel price scenarios
fuel_multipliers = [0.8, 1.0, 1.2]  # 20% decrease, no change, 20% increase
results = []

for multiplier in fuel_multipliers:
    total_cost = solve_problem(multiplier)
    total_gallons_required = sum(gallons_required)
    breakeven_price_per_gallon = total_cost / total_gallons_required
    bid_price_per_gallon = breakeven_price_per_gallon * 1.15
    results.append({
        "Fuel Multiplier": multiplier,
        "Total Cost": total_cost,
        "Breakeven Price per Gallon": breakeven_price_per_gallon,
        "Bid Price per Gallon": bid_price_per_gallon
    })

# Print the results
print("Results for different fuel price scenarios:")
for result in results:
    print(f"Fuel Multiplier: {result['Fuel Multiplier']}")
    print(f"Total Cost: {result['Total Cost']:.2f} USD")
    print(f"Breakeven Price per Gallon: {result['Breakeven Price per Gallon']:.2f} USD")
    print(f"Bid Price per Gallon: {result['Bid Price per Gallon']:.2f} USD")
    print()

Results for different fuel price scenarios:
Fuel Multiplier: 0.8
Total Cost: 1055187.94 USD
Breakeven Price per Gallon: 1.36 USD
Bid Price per Gallon: 1.57 USD

Fuel Multiplier: 1.0
Total Cost: 1318984.93 USD
Breakeven Price per Gallon: 1.71 USD
Bid Price per Gallon: 1.96 USD

Fuel Multiplier: 1.2
Total Cost: 1582781.92 USD
Breakeven Price per Gallon: 2.05 USD
Bid Price per Gallon: 2.35 USD



In [77]:
import numpy as np
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, value

# Data from the problem
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City", "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"]
gallons_required = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]
cincinnati_cost = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_cost = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

# Production costs
cincinnati_production_cost = 1.20
oakland_production_cost = 1.65

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define the decision variables
x = LpVariable.dicts("Route", [(i,j) for i in range(2) for j in range(11)], lowBound=0)

# Define the cost matrix (production + shipping costs)
def calculate_total_costs(fuel_multiplier):
    cincinnati_total_cost = [(c + cincinnati_production_cost) * fuel_multiplier for c in cincinnati_cost]
    oakland_total_cost = [(o + oakland_production_cost) * fuel_multiplier for o in oakland_cost]
    return [cincinnati_total_cost, oakland_total_cost]

# Define the supply and demand constraints
supply = [500000, 500000]  # Assuming a capacity of 500,000 gallons for each plant

# Function to solve the problem and calculate costs
def solve_problem(fuel_multiplier):
    cost = calculate_total_costs(fuel_multiplier)
    prob = LpProblem("TransportationProblem", LpMinimize)
    prob += lpSum([cost[i][j] * x[i,j] for i in range(2) for j in range(11)])
    for i in range(2):
        prob += lpSum([x[i,j] for j in range(11)]) <= supply[i]
    for j in range(11):
        prob += lpSum([x[i,j] for i in range(2)]) >= gallons_required[j]
    prob.solve()
    total_costs = []
    for j in range(11):
        gallons_cincinnati = value(x[0,j])
        gallons_oakland = value(x[1,j])
        cost_cincinnati = gallons_cincinnati * cost[0][j]
        cost_oakland = gallons_oakland * cost[1][j]
        total_location_cost = cost_cincinnati + cost_oakland
        total_costs.append(total_location_cost)
    total_cost = sum(total_costs)
    return total_cost

# Simulate different fuel price scenarios
fuel_multipliers = [0.6,0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4]  # Adding more multipliers to see better variance of the results
results = []

for multiplier in fuel_multipliers:
    total_cost = solve_problem(multiplier)
    total_gallons_required = sum(gallons_required)
    breakeven_price_per_gallon = total_cost / total_gallons_required
    bid_price_per_gallon = breakeven_price_per_gallon * 1.15
    results.append({
        "Fuel Multiplier": multiplier,
        "Total Cost": total_cost,
        "Breakeven Price per Gallon": breakeven_price_per_gallon,
        "Bid Price per Gallon": bid_price_per_gallon
    })

# Print the results
print("Results for different fuel price scenarios:")
for result in results:
    print(f"Fuel Multiplier: {result['Fuel Multiplier']}")
    print(f"Total Cost: {result['Total Cost']:.2f} USD")
    print(f"Breakeven Price per Gallon: {result['Breakeven Price per Gallon']:.2f} USD")
    print(f"Bid Price per Gallon: {result['Bid Price per Gallon']:.2f} USD")
    print()

Results for different fuel price scenarios:
Fuel Multiplier: 0.6
Total Cost: 791390.96 USD
Breakeven Price per Gallon: 1.02 USD
Bid Price per Gallon: 1.18 USD

Fuel Multiplier: 0.7
Total Cost: 923289.45 USD
Breakeven Price per Gallon: 1.19 USD
Bid Price per Gallon: 1.37 USD

Fuel Multiplier: 0.8
Total Cost: 1055187.94 USD
Breakeven Price per Gallon: 1.36 USD
Bid Price per Gallon: 1.57 USD

Fuel Multiplier: 0.9
Total Cost: 1187086.44 USD
Breakeven Price per Gallon: 1.53 USD
Bid Price per Gallon: 1.76 USD

Fuel Multiplier: 1.0
Total Cost: 1318984.93 USD
Breakeven Price per Gallon: 1.71 USD
Bid Price per Gallon: 1.96 USD

Fuel Multiplier: 1.1
Total Cost: 1450883.42 USD
Breakeven Price per Gallon: 1.88 USD
Bid Price per Gallon: 2.16 USD

Fuel Multiplier: 1.2
Total Cost: 1582781.92 USD
Breakeven Price per Gallon: 2.05 USD
Bid Price per Gallon: 2.35 USD

Fuel Multiplier: 1.3
Total Cost: 1714680.41 USD
Breakeven Price per Gallon: 2.22 USD
Bid Price per Gallon: 2.55 USD

Fuel Multiplier: 1.4
T

##2e

In [72]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, value

# Data from the problem
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City", "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"]
gallons_required = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]
cincinnati_cost = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_cost = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

# Production costs
cincinnati_production_cost = 1.20
oakland_production_cost = 1.65

# Define the scenarios
scenarios = [
    {"production_multiplier": 1.1, "shipping_multiplier": 1.1}, # Increase both by 10%
    {"production_multiplier": 1.1, "shipping_multiplier": 1.0}, # Increase production by 10%
    {"production_multiplier": 1.0, "shipping_multiplier": 1.1}, # Increase shipping by 10%
    {"production_multiplier": 1.0, "shipping_multiplier": 1.0}, # Nothing changes
    {"production_multiplier": 0.9, "shipping_multiplier": 0.9}, # Decrease both by 10%
    {"production_multiplier": 0.9, "shipping_multiplier": 1.0}, # Decrease production by 10%
    {"production_multiplier": 1.0, "shipping_multiplier": 0.9}, # Decrease shipping by 10%
    {"production_multiplier": 1.1, "shipping_multiplier": 0.9}, # Increase production by 10% and decrease shipping by 10%
    {"production_multiplier": 0.9, "shipping_multiplier": 1.1}, # Decrease production by 10% and increase shipping by 10%
]

# Function to solve the problem for a given scenario
def solve_scenario(production_multiplier, shipping_multiplier):
    # Adjust costs based on multipliers
    adjusted_cincinnati_cost = [c * shipping_multiplier for c in cincinnati_cost]
    adjusted_oakland_cost = [o * shipping_multiplier for o in oakland_cost]

    adjusted_cincinnati_production_cost = cincinnati_production_cost * production_multiplier
    adjusted_oakland_production_cost = oakland_production_cost * production_multiplier

    cincinnati_total_cost = [c + adjusted_cincinnati_production_cost for c in adjusted_cincinnati_cost]
    oakland_total_cost = [o + adjusted_oakland_production_cost for o in adjusted_oakland_cost]

    # Define the problem
    prob = LpProblem("Minimise_Transportaion_Cost", LpMinimize)

    # Define the decision variables
    x = LpVariable.dicts("Route", [(i,j) for i in range(2) for j in range(11)], lowBound=0)

    # Define the cost matrix (production + shipping costs)
    cost = [cincinnati_total_cost, oakland_total_cost]

    # Define the supply and demand constraints
    supply = [500000, 500000]

    # Objective function: Minimize total cost
    prob += lpSum([cost[i][j] * x[i,j] for i in range(2) for j in range(11)])

    # Supply constraints
    for i in range(2):
        prob += lpSum([x[i,j] for j in range(11)]) <= supply[i]

    # Demand constraints
    for j in range(11):
        prob += lpSum([x[i,j] for i in range(2)]) >= gallons_required[j]

    # Solve the problem
    prob.solve()

    # Extract results
    results = []
    total_gallons_cincinnati = total_gallons_oakland = total_cost = 0

    for j in range(11):
        gallons_cincinnati = value(x[0,j])
        gallons_oakland = value(x[1,j])
        total_gallons = gallons_cincinnati + gallons_oakland
        cost_cincinnati = gallons_cincinnati * cincinnati_total_cost[j]
        cost_oakland = gallons_oakland * oakland_total_cost[j]
        total_location_cost = cost_cincinnati + cost_oakland

        results.append({
            "Location": locations[j],
            "From Cincinnati": gallons_cincinnati,
            "From Oakland": gallons_oakland,
            "Total Gallons": total_gallons,
            "Cost": total_location_cost
        })

        total_gallons_cincinnati += gallons_cincinnati
        total_gallons_oakland += gallons_oakland
        total_cost += total_location_cost

    return results, total_gallons_cincinnati, total_gallons_oakland, total_cost

# Analyze each scenario and print the results
for scenario in scenarios:
    results, total_gallons_cincinnati, total_gallons_oakland, total_cost = solve_scenario(scenario["production_multiplier"], scenario["shipping_multiplier"])

    print(f"Scenario: Production Multiplier={scenario['production_multiplier']}, Shipping Multiplier={scenario['shipping_multiplier']}")
    print(f"{'Location':<15} {'From Cincinnati':<20} {'From Oakland':<15} {'Total Gallons':<15} {'Cost':<10}")

    for result in results:
        print(f"{result['Location']:<15} {result['From Cincinnati']:<20} {result['From Oakland']:<15} {result['Total Gallons']:<15} {result['Cost']:<10.2f}")

    print(f"\n{'Total':<15} {total_gallons_cincinnati:<20} {total_gallons_oakland:<15} {total_gallons_cincinnati + total_gallons_oakland:<15} {total_cost:<10.2f}")
    print("\n" + "-"*80 + "\n")

Scenario: Production Multiplier=1.1, Shipping Multiplier=1.1
Location        From Cincinnati      From Oakland    Total Gallons   Cost      
Santa Ana       0.0                  22418.0         22418.0         46113.83  
El Paso         6800.0               0.0             6800.0          15259.20  
Pendleton       39636.0              40654.0         80290.0         184206.70 
Houston         100447.0             0.0             100447.0        182311.31 
Kansas City     24570.0              0.0             24570.0         42162.12  
Los Angeles     0.0                  64761.0         64761.0         133213.38 
Glendale        0.0                  33689.0         33689.0         69298.27  
Jacksonville    68486.0              0.0             68486.0         116015.28 
Little Rock     148586.0             0.0             148586.0        251704.68 
Bridgeport      111475.0             0.0             111475.0        188838.65 
Sacramento      0.0                  112000.0        112000

# Question 3 (complete)

##3a

In [32]:
# Intall and Import Package/Library
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils
from pulp import GLPK

# Define the problem
prob = pulp.LpProblem("Economic_Goals", pulp.LpMinimize)

# Define decision variables
G = pulp.LpVariable("G", lowBound=0, cat='Continuous')
T1 = pulp.LpVariable("T1", lowBound=0, cat='Continuous')
T2 = pulp.LpVariable("T2", lowBound=0, cat='Continuous')
C = pulp.LpVariable("C", lowBound=0, upBound=150, cat='Continuous')

# Define deviation variables
d1_pos = pulp.LpVariable("d1_pos", lowBound=0, cat='Continuous')
d1_neg = pulp.LpVariable("d1_neg", lowBound=0, cat='Continuous')
d2_pos = pulp.LpVariable("d2_pos", lowBound=0, cat='Continuous')
d2_neg = pulp.LpVariable("d2_neg", lowBound=0, cat='Continuous')
d3_pos = pulp.LpVariable("d3_pos", lowBound=0, cat='Continuous')
d3_neg = pulp.LpVariable("d3_neg", lowBound=0, cat='Continuous')
d4_pos = pulp.LpVariable("d4_pos", lowBound=0, cat='Continuous')
d4_neg = pulp.LpVariable("d4_neg", lowBound=0, cat='Continuous')

# Add objective function to minimize deviations
#prob += d1_pos + d1_neg + d2_pos + d2_neg + d3_pos + d3_neg + d4_pos + d4_neg, "Objective"
prob += d1_neg +d2_pos + d3_pos + d4_pos, "Objective"

# Add constraints
# Goal 1: Balance the budget (revenues >= costs)
prob += 1.5*G + 25*T1 + 15*T2 +C - d1_pos + d1_neg == 1000
# Goal 2: Cut spending by at most $150 billion
prob += C - d2_pos + d2_neg == 150

# Goal 3: Raise at most $550 billion in taxes from the upper class
prob += 0.5*G + 5*T1 + 15 * T2 - d3_pos + d3_neg == 550

# Goal 4: Raise at most $350 billion in taxes from the lower class
prob += G + 20 * T1 - d4_pos + d4_neg == 350

# Tax rate condition: T2 >= T1
prob += T2 >= T1

# Solve the problem
prob.solve()

# Print the results
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"G (Gas tax rate): {G.varValue} cents")
print(f"T1 (Tax rate on first $30,000): {T1.varValue}")
print(f"T2 (Tax rate on income over $30,000): {T2.varValue}")
print(f"C (Spending cut): {C.varValue} billion dollars")
print(f"Deviation from Goal 1: Positive {d1_pos.varValue}, Negative {d1_neg.varValue}")
print(f"Deviation from Goal 2: Positive {d2_pos.varValue}, Negative {d2_neg.varValue}")
print(f"Deviation from Goal 3: Positive {d3_pos.varValue}, Negative {d3_neg.varValue}")
print(f"Deviation from Goal 4: Positive {d4_pos.varValue}, Negative {d4_neg.varValue}")

# Calculate total revenue
total_revenue = 1.5 * G.varValue + 25 * T1.varValue + 15 * T2.varValue
print(f"Total Revenue: {total_revenue} billion dollars")

Status: Optimal
G (Gas tax rate): 300.0 cents
T1 (Tax rate on first $30,000): 0.0
T2 (Tax rate on income over $30,000): 26.666667
C (Spending cut): 150.0 billion dollars
Deviation from Goal 1: Positive 0.0, Negative 0.0
Deviation from Goal 2: Positive 0.0, Negative 0.0
Deviation from Goal 3: Positive 0.0, Negative 0.0
Deviation from Goal 4: Positive 0.0, Negative 50.0
Total Revenue: 850.000005 billion dollars


##3b


In [33]:
import numpy as np

# Define the sensitivity analysis function
def sensitivity_analysis(G, T1, T2):
    # Define the range of changes for each variable
    delta = np.linspace(-0.1, 0.1, 21)

    # Initialize lists to store results
    revenue_G = []
    revenue_T1 = []
    revenue_T2 = []

    # Calculate revenue for changes in G
    for d in delta:
        new_G = G * (1 + d)
        revenue = 1.5 * new_G + 25 * T1 + 15 * T2
        revenue_G.append(revenue)

    # Calculate revenue for changes in T1
    for d in delta:
        new_T1 = T1 * (1 + d)
        revenue = 1.5 * G + 25 * new_T1 + 15 * T2
        revenue_T1.append(revenue)

    # Calculate revenue for changes in T2
    for d in delta:
        new_T2 = T2 * (1 + d)
        revenue = 1.5 * G + 25 * T1 + 15 * new_T2
        revenue_T2.append(revenue)

    return delta, revenue_G, revenue_T1, revenue_T2

# Optimal values from the previous solution
G_opt = 350
T1_opt = 0
T2_opt = 21.66667

# Conduct sensitivity analysis
delta, revenue_G, revenue_T1, revenue_T2 = sensitivity_analysis(G_opt, T1_opt, T2_opt)

# Print the results
print("Sensitivity Analysis Results:")
print("Changes in G:")
for d, rev in zip(delta, revenue_G):
    print(f"Change: {d*100:.1f}%, Revenue: {rev:.2f} billion dollars")

print("\nChanges in T1:")
for d, rev in zip(delta, revenue_T1):
    print(f"Change: {d*100:.1f}%, Revenue: {rev:.2f} billion dollars")

print("\nChanges in T2:")
for d, rev in zip(delta, revenue_T2):
    print(f"Change: {d*100:.1f}%, Revenue: {rev:.2f} billion dollars")

# Determine which variable has the most significant impact on total revenue
impact_G = max(revenue_G) - min(revenue_G)
impact_T1 = max(revenue_T1) - min(revenue_T1)
impact_T2 = max(revenue_T2) - min(revenue_T2)

most_significant_impact = max(impact_G, impact_T1, impact_T2)
if most_significant_impact == impact_G:
    most_significant_variable = "G"
elif most_significant_impact == impact_T1:
    most_significant_variable = "T1"
else:
    most_significant_variable = "T2"

print(f"\nThe variable with the most significant impact on total revenue is: {most_significant_variable}")


Sensitivity Analysis Results:
Changes in G:
Change: -10.0%, Revenue: 797.50 billion dollars
Change: -9.0%, Revenue: 802.75 billion dollars
Change: -8.0%, Revenue: 808.00 billion dollars
Change: -7.0%, Revenue: 813.25 billion dollars
Change: -6.0%, Revenue: 818.50 billion dollars
Change: -5.0%, Revenue: 823.75 billion dollars
Change: -4.0%, Revenue: 829.00 billion dollars
Change: -3.0%, Revenue: 834.25 billion dollars
Change: -2.0%, Revenue: 839.50 billion dollars
Change: -1.0%, Revenue: 844.75 billion dollars
Change: 0.0%, Revenue: 850.00 billion dollars
Change: 1.0%, Revenue: 855.25 billion dollars
Change: 2.0%, Revenue: 860.50 billion dollars
Change: 3.0%, Revenue: 865.75 billion dollars
Change: 4.0%, Revenue: 871.00 billion dollars
Change: 5.0%, Revenue: 876.25 billion dollars
Change: 6.0%, Revenue: 881.50 billion dollars
Change: 7.0%, Revenue: 886.75 billion dollars
Change: 8.0%, Revenue: 892.00 billion dollars
Change: 9.0%, Revenue: 897.25 billion dollars
Change: 10.0%, Revenue: 9

# Question 4 (complete)

##4a

In [42]:
import numpy as np
from scipy.optimize import linprog

# Define the transportation costs from production centres to warehouses
cost_pc_wh = np.array([
    [4, 3, 5], #Production A
    [6, 2, 4], #Production B
    [5, 4, 3], #Production C
    [7, 5, 6]  #Production D
])

# Define the transportation costs from warehouses to distribution centres
cost_wh_dc = np.array([
    [3, 2, 5, 4, 6, 7],
    [1, 4, 2, 3, 5, 6],
    [5, 3, 4, 2, 3, 4]
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc.shape[0]

# Create the cost vector for the linear programming problem
cost_vector = np.concatenate([cost_pc_wh.flatten(), cost_wh_dc.flatten()])

# Create the equality constraints matrix and vector
A_eq = []
b_eq = []

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    constraint[i * num_wh:(i + 1) * num_wh] = 1
    A_eq.append(constraint)
    b_eq.append(supply_pc[i])

# Demand constraints for warehouses to distribution centres
for j in range(num_dc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for k in range(num_wh):
        constraint[num_pc * num_wh + k * num_dc + j] = -1
    A_eq.append(constraint)
    b_eq.append(-demand_dc[j])

# Flow conservation constraints for warehouses
for k in range(num_wh):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for i in range(num_pc):
        constraint[i * num_wh + k] = -1
    for j in range(num_dc):
        constraint[num_pc * num_wh + k * num_dc + j] = 1
    A_eq.append(constraint)
    b_eq.append(0)

A_eq = np.array(A_eq)
b_eq = np.array(b_eq)

# Bounds for the decision variables (all variables must be non-negative)
bounds = [(0, None) for _ in range(num_pc * num_wh + num_wh * num_dc)]

# Solve the linear programming problem
result = linprog(cost_vector, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Extract the optimal solution
solution = result.x

# Reshape the solution to get the flows from production centres to warehouses and from warehouses to distribution centres
flow_pc_wh = solution[:num_pc * num_wh].reshape((num_pc, num_wh))
flow_wh_dc = solution[num_pc * num_wh:].reshape((num_wh, num_dc))

# Format the output to avoid negative zeros by replacing them with positive zeros
flow_pc_wh[flow_pc_wh == -0.0] = 0.0
flow_wh_dc[flow_wh_dc == -0.0] = 0.0

print("Optimal flows from production centres to warehouses:")
print(flow_pc_wh)

print("Optimal flows from warehouses to distribution centres:")
print(flow_wh_dc)

print("Optimal transportation cost:")
print(result.fun)

Optimal flows from production centres to warehouses:
[[40. 30.  0.]
 [ 0. 90.  0.]
 [ 0.  0. 80.]
 [ 0. 60.  0.]]
Optimal flows from warehouses to distribution centres:
[[ 0. 40.  0.  0.  0.  0.]
 [50.  0. 60. 70.  0.  0.]
 [ 0.  0.  0.  0. 50. 30.]]
Optimal transportation cost:
1700.0


##4b

In [46]:
import numpy as np
from scipy.optimize import linprog

# Define the transportation costs from production centres to warehouses with increased costs for Warehouse 2 (middle)
cost_pc_wh = np.array([
    [4, 4, 5],  # Increased cost by £1
    [6, 3, 4],  # Increased cost by £1
    [5, 5, 3],  # Increased cost by £1
    [7, 6, 6]   # Increased cost by £1
])

# Define the transportation costs from warehouses to distribution centres with increased costs for Warehouse 2
cost_wh_dc = np.array([
    [3, 2, 5, 4, 6, 7],
    [2, 5, 3, 4, 6, 7],  # Increased cost by £1
    [5, 3, 4, 2, 3, 4]
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc.shape[0]

# Create the cost vector for the linear programming problem
cost_vector = np.concatenate([cost_pc_wh.flatten(), cost_wh_dc.flatten()])

# Create the equality constraints matrix and vector
A_eq = []
b_eq = []

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    constraint[i * num_wh:(i + 1) * num_wh] = 1
    A_eq.append(constraint)
    b_eq.append(supply_pc[i])

# Demand constraints for warehouses to distribution centres
for j in range(num_dc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for k in range(num_wh):
        constraint[num_pc * num_wh + k * num_dc + j] = -1
    A_eq.append(constraint)
    b_eq.append(-demand_dc[j])

# Flow conservation constraints for warehouses
for k in range(num_wh):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for i in range(num_pc):
        constraint[i * num_wh + k] = -1
    for j in range(num_dc):
        constraint[num_pc * num_wh + k * num_dc + j] = 1
    A_eq.append(constraint)
    b_eq.append(0)

A_eq = np.array(A_eq)
b_eq = np.array(b_eq)

# Bounds for the decision variables (all variables must be non-negative)
bounds = [(0, None) for _ in range(num_pc * num_wh + num_wh * num_dc)]

# Solve the linear programming problem
result = linprog(cost_vector, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Extract the optimal solution
solution = result.x

# Reshape the solution to get the flows from production centres to warehouses and from warehouses to distribution centres
flow_pc_wh = solution[:num_pc * num_wh].reshape((num_pc, num_wh))
flow_wh_dc = solution[num_pc * num_wh:].reshape((num_wh, num_dc))

print("Optimal flows from production centres to warehouses:")
print(flow_pc_wh)

print("Optimal flows from warehouses to distribution centres:")
print(flow_wh_dc)

print("Optimal transportation cost:")
print(result.fun)

Optimal flows from production centres to warehouses:
[[40. 20. 10.]
 [ 0. 90.  0.]
 [ 0.  0. 80.]
 [ 0.  0. 60.]]
Optimal flows from warehouses to distribution centres:
[[ 0. 40.  0.  0.  0.  0.]
 [50.  0. 60.  0.  0.  0.]
 [ 0.  0.  0. 70. 50. 30.]]
Optimal transportation cost:
1930.0


##4c

In [63]:
import numpy as np
from scipy.optimize import linprog

# Define the transportation costs from production centres to warehouses
cost_pc_wh = np.array([
    [4, 3, 5], #Production A
    [6, 2, 4], #Production B
    [5, 4, 3], #Production C
    [7, 5, 6]  #Production D
])

# Define the transportation costs from warehouses to distribution centres
cost_wh_dc = np.array([
    [3, 2, 5, 4, 6, 7],
    [1, 4, 2, 3, 5, 6],
    [5, 3, 4, 2, 3, 4]
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc.shape[0]

# Create the cost vector for the linear programming problem
cost_vector = np.concatenate([cost_pc_wh.flatten(), cost_wh_dc.flatten()])

# Create the equality constraints matrix and vector
A_eq = []
b_eq = []

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    constraint[i * num_wh:(i + 1) * num_wh] = 1
    A_eq.append(constraint)
    b_eq.append(supply_pc[i])

# Demand constraints for warehouses to distribution centres
for j in range(num_dc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for k in range(num_wh):
        constraint[num_pc * num_wh + k * num_dc + j] = -1
    A_eq.append(constraint)
    b_eq.append(-demand_dc[j])

# Flow conservation constraints for warehouses
for k in range(num_wh):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for i in range(num_pc):
        constraint[i * num_wh + k] = -1
    for j in range(num_dc):
        constraint[num_pc * num_wh + k * num_dc + j] = 1
    A_eq.append(constraint)
    b_eq.append(0)

A_eq = np.array(A_eq)
b_eq = np.array(b_eq)

# Bounds for the decision variables (all variables must be non-negative)
bounds = [(0, None) for _ in range(num_pc * num_wh + num_wh * num_dc)]

# Add the constraint that the handling capacity of Warehouse 1 does not exceed 50 units
A_ub = np.zeros((1, num_pc * num_wh + num_wh * num_dc))
for i in range(num_pc):
    A_ub[0][i * num_wh + 0] = 1

b_ub = [50]

# Solve the linear programming problem with the new constraint
result = linprog(cost_vector, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Extract the optimal solution
solution = result.x

# Reshape the solution to get the flows from production centres to warehouses and from warehouses to distribution centres
flow_pc_wh = solution[:num_pc * num_wh].reshape((num_pc, num_wh))
flow_wh_dc = solution[num_pc * num_wh:].reshape((num_wh, num_dc))

# Format the output to avoid negative zeros by replacing them with positive zeros
flow_pc_wh[flow_pc_wh == -0.0] = 0.0
flow_wh_dc[flow_wh_dc == -0.0] = 0.0

print("Optimal flows from production centres to warehouses:")
print(flow_pc_wh)

print("Optimal flows from warehouses to distribution centres:")
print(flow_wh_dc)

print("Optimal transportation cost:")
print(result.fun)


Optimal flows from production centres to warehouses:
[[40. 30.  0.]
 [ 0. 90.  0.]
 [ 0.  0. 80.]
 [ 0.  0. 60.]]
Optimal flows from warehouses to distribution centres:
[[ 0. 40.  0.  0.  0.  0.]
 [50.  0. 60. 10.  0.  0.]
 [ 0.  0.  0. 60. 50. 30.]]
Optimal transportation cost:
1700.0


##4d

In [59]:
import numpy as np
from scipy.optimize import linprog

# Define the transportation costs from production centres to warehouses
cost_pc_wh = np.array([
    [4, 3, 5],
    [6, 2, 4],
    [5, 4, 3],
    [7, 5, 6]
])

# Define the transportation costs from warehouses to distribution centres
cost_wh_dc = np.array([
    [3, 2, 5, 4, 6, 7],
    [1, 4, 2, 3, 5, 6],
    [5, 3, 4, 2, 3, 4]
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Remove DC1 and equally assign its demand to others
demand_dc_new = np.array([0, 40 + (50/5), 60 + (50/5), 70 + (50/5), 50 + (50/5), 30 + (50/5)])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc_new.shape[0]

# Create the cost vector for the linear programming problem
cost_vector = np.concatenate([cost_pc_wh.flatten(), cost_wh_dc.flatten()])

# Create the equality constraints matrix and vector
A_eq = []
b_eq = []

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    constraint[i * num_wh:(i + 1) * num_wh] = 1
    A_eq.append(constraint)
    b_eq.append(supply_pc[i])

# Demand constraints for warehouses to distribution centres
for j in range(num_dc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for k in range(num_wh):
        constraint[num_pc * num_wh + k * num_dc + j] = -1
    A_eq.append(constraint)
    b_eq.append(-demand_dc_new[j])

# Flow conservation constraints for warehouses
for k in range(num_wh):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for i in range(num_pc):
        constraint[i * num_wh + k] = -1
    for j in range(num_dc):
        constraint[num_pc * num_wh + k * num_dc + j] = 1
    A_eq.append(constraint)
    b_eq.append(0)

A_eq = np.array(A_eq)
b_eq = np.array(b_eq)

# Bounds for the decision variables (all variables must be non-negative)
bounds = [(0, None) for _ in range(num_pc * num_wh + num_wh * num_dc)]

# Solve the linear programming problem
result = linprog(cost_vector, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Extract the optimal solution
solution = result.x

# Reshape the solution to get the flows from production centres to warehouses and from warehouses to distribution centres
flow_pc_wh = solution[:num_pc * num_wh].reshape((num_pc, num_wh))
flow_wh_dc = solution[num_pc * num_wh:].reshape((num_wh, num_dc))

print("Optimal flows from production centres to warehouses:")
print(flow_pc_wh)

print("Optimal flows from warehouses to distribution centres:")
print(flow_wh_dc)

print("Optimal transportation cost:")
print(result.fun)

Optimal flows from production centres to warehouses:
[[50. 20.  0.]
 [ 0. 90.  0.]
 [ 0.  0. 80.]
 [ 0. 40. 20.]]
Optimal flows from warehouses to distribution centres:
[[ 0. 50.  0.  0.  0.  0.]
 [ 0.  0. 70. 80.  0.  0.]
 [ 0.  0.  0.  0. 60. 40.]]
Optimal transportation cost:
1820.0


In [60]:
import numpy as np
from scipy.optimize import linprog

# Define the transportation costs from production centres to warehouses
cost_pc_wh = np.array([
    [4, 3, 5],
    [6, 2, 4],
    [5, 4, 3],
    [7, 5, 6]
])

# Define the transportation costs from warehouses to distribution centres
cost_wh_dc = np.array([
    [3, 2, 5, 4, 6, 7],
    [1, 4, 2, 3, 5, 6],
    [5, 3, 4, 2, 3, 4]
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Remove DC5 and equally assign its demand to others
demand_dc_new = np.array([50 + (50/5), 40 + (50/5), 60 + (50/5), 70 + (50/5), 0, 30 + (50/5)])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc_new.shape[0]

# Create the cost vector for the linear programming problem
cost_vector = np.concatenate([cost_pc_wh.flatten(), cost_wh_dc.flatten()])

# Create the equality constraints matrix and vector
A_eq = []
b_eq = []

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    constraint[i * num_wh:(i + 1) * num_wh] = 1
    A_eq.append(constraint)
    b_eq.append(supply_pc[i])

# Demand constraints for warehouses to distribution centres
for j in range(num_dc):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for k in range(num_wh):
        constraint[num_pc * num_wh + k * num_dc + j] = -1
    A_eq.append(constraint)
    b_eq.append(-demand_dc_new[j])

# Flow conservation constraints for warehouses
for k in range(num_wh):
    constraint = np.zeros(num_pc * num_wh + num_wh * num_dc)
    for i in range(num_pc):
        constraint[i * num_wh + k] = -1
    for j in range(num_dc):
        constraint[num_pc * num_wh + k * num_dc + j] = 1
    A_eq.append(constraint)
    b_eq.append(0)

A_eq = np.array(A_eq)
b_eq = np.array(b_eq)

# Bounds for the decision variables (all variables must be non-negative)
bounds = [(0, None) for _ in range(num_pc * num_wh + num_wh * num_dc)]

# Solve the linear programming problem
result = linprog(cost_vector, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Extract the optimal solution
solution = result.x

# Reshape the solution to get the flows from production centres to warehouses and from warehouses to distribution centres
flow_pc_wh = solution[:num_pc * num_wh].reshape((num_pc, num_wh))
flow_wh_dc = solution[num_pc * num_wh:].reshape((num_wh, num_dc))

print("Optimal flows from production centres to warehouses:")
print(flow_pc_wh)

print("Optimal flows from warehouses to distribution centres:")
print(flow_wh_dc)

print("Total transportation cost:")
print(result.fun)

Optimal flows from production centres to warehouses:
[[50. 20.  0.]
 [ 0. 90.  0.]
 [ 0.  0. 80.]
 [ 0. 60.  0.]]
Optimal flows from warehouses to distribution centres:
[[ 0. 50.  0.  0.  0.  0.]
 [60.  0. 70. 40.  0.  0.]
 [ 0.  0.  0. 40.  0. 40.]]
Total transportation cost:
1640.0
